<a href="https://colab.research.google.com/github/hvarS/NLPRefer/blob/main/UnivariateTimeSeriesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [163]:
df = pd.read_csv("train.csv")

In [164]:
df.head()

,Usage,Timestep,InventoryCode,Domain
0,93,1,0,B
1,94,2,0,B
2,94,3,0,B
3,94,4,0,B
4,94,5,0,B


In [165]:
df["Domain"].unique()

array(['B', 'C[ii]', 'C[i]', 'A'], dtype=object)

In [166]:
df["InventoryCode"].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [235]:
df.loc[(df["Domain"]=='B') & (df["InventoryCode"]==0)]

,Usage,Timestep,InventoryCode,Domain
0,93,1,0,B
1,94,2,0,B
2,94,3,0,B
3,94,4,0,B
4,94,5,0,B
...,...,...,...,...
89,36,90,0,B
90,36,91,0,B
91,36,92,0,B
92,37,93,0,B


In [168]:
##Creating a dictionary to store timesteps according to domain , inventory
data = {}
domains = df["Domain"].unique()
inventories = df["InventoryCode"].unique()
for domain in domains:
  data[domain] = {}
  for inventory in inventories:
    data[domain][inventory] = df.loc[(df["Domain"]==domain) & (df["InventoryCode"]==inventory)]

##Data Preprocessing 

In [169]:
def preprocess(timeseries_data,n_features):
  x,y = [],[]
  series_data = list(timeseries_data["Usage"])
  for i in range(len(timeseries_data)):
    end_idx = i+n_features
    print(end_idx)
    if end_idx > len(series_data)-1:
      break
    else:
      seq_x,seq_y = series_data[i:end_idx],series_data[end_idx]
      x.append(np.array(seq_x))
      y.append(seq_y)
  return np.array(x),np.array(y)

In [170]:
n_features = {"A":100,"B":75,"C[i]":35,"C[ii]":35}

In [ ]:
for domain in domains:
  for inventory in inventories:
    req_features = n_features[domain]
    print(domain,inventory)
    x,y = preprocess(data[domain][inventory],req_features)
    data[domain][inventory] = [x,y]

In [12]:
df_a_x = data["A"][inventory][0]
df_a_y = data["A"][inventory][1]
for i in range(1,len(inventories)):
  df_a_x = np.concatenate((df_a_x,data["A"][i][0]),axis = 0)
  df_a_y = np.concatenate((df_a_y,data["A"][i][1]),axis = 0)

In [13]:
df_b_x = data["B"][inventory][0]
df_b_y = data["B"][inventory][1]
for i in range(1,len(inventories)):
  df_b_x = np.concatenate((df_b_x,data["B"][i][0]),axis = 0)
  df_b_y = np.concatenate((df_b_y,data["B"][i][1]),axis = 0)

In [14]:
df_ci_x = data["C[i]"][inventory][0]
df_ci_y = data["C[i]"][inventory][1]
for i in range(1,len(inventories)):
  if len(data["C[i]"][i][0]) == 0:
    break
  df_ci_x = np.concatenate((df_ci_x,data["C[i]"][i][0]),axis = 0)
  df_ci_y = np.concatenate((df_ci_y,data["C[i]"][i][1]),axis = 0)

ValueError: ignored

In [ ]:
df_cii_x = data["C[ii]"][inventory][0]
df_cii_y = data["C[ii]"][inventory][1]
for i in range(1,len(inventories)):
  if len(data["C[i]"][i][0]) == 0:
    break
  df_cii_x = np.concatenate((df_cii_x,data["C[ii]"][i][0]),axis = 0)
  df_cii_y = np.concatenate((df_cii_y,data["C[ii]"][i][1]),axis = 0)

In [15]:
df_a = pd.DataFrame(df_a_x)

In [16]:
df_a["Target"]  = df_a_y

In [17]:
df_a

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,Target
0,13,27,28,30,30,30,30,13,30,30,30,31,35,35,14,31,31,32,34,34,32,15,30,30,25,28,28,28,12,29,29,27,27,27,26,14,26,28,28,29,...,35,35,16,33,33,33,33,32,29,16,29,33,33,30,29,28,14,29,29,30,30,30,30,14,15,30,30,31,31,30,14,31,33,33,33,34,35,16,34,34
1,27,28,30,30,30,30,13,30,30,30,31,35,35,14,31,31,32,34,34,32,15,30,30,25,28,28,28,12,29,29,27,27,27,26,14,26,28,28,29,29,...,35,16,33,33,33,33,32,29,16,29,33,33,30,29,28,14,29,29,30,30,30,30,14,15,30,30,31,31,30,14,31,33,33,33,34,35,16,34,34,34
2,28,30,30,30,30,13,30,30,30,31,35,35,14,31,31,32,34,34,32,15,30,30,25,28,28,28,12,29,29,27,27,27,26,14,26,28,28,29,29,29,...,16,33,33,33,33,32,29,16,29,33,33,30,29,28,14,29,29,30,30,30,30,14,15,30,30,31,31,30,14,31,33,33,33,34,35,16,34,34,34,35
3,30,30,30,30,13,30,30,30,31,35,35,14,31,31,32,34,34,32,15,30,30,25,28,28,28,12,29,29,27,27,27,26,14,26,28,28,29,29,29,14,...,33,33,33,33,32,29,16,29,33,33,30,29,28,14,29,29,30,30,30,30,14,15,30,30,31,31,30,14,31,33,33,33,34,35,16,34,34,34,35,37
4,30,30,30,13,30,30,30,31,35,35,14,31,31,32,34,34,32,15,30,30,25,28,28,28,12,29,29,27,27,27,26,14,26,28,28,29,29,29,14,28,...,33,33,33,32,29,16,29,33,33,30,29,28,14,29,29,30,30,30,30,14,15,30,30,31,31,30,14,31,33,33,33,34,35,16,34,34,34,35,37,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,31,31,32,34,34,32,15,30,30,25,28,28,28,12,29,29,27,27,27,26,14,26,28,28,29,29,29,14,28,31,33,33,33,32,15,33,34,34,34,34,...,28,14,29,29,30,30,30,30,14,15,30,30,31,31,30,14,31,33,33,33,34,35,16,34,34,34,35,37,37,16,31,31,32,32,33,33,16,34,34,35
4996,31,32,34,34,32,15,30,30,25,28,28,28,12,29,29,27,27,27,26,14,26,28,28,29,29,29,14,28,31,33,33,33,32,15,33,34,34,34,34,34,...,14,29,29,30,30,30,30,14,15,30,30,31,31,30,14,31,33,33,33,34,35,16,34,34,34,35,37,37,16,31,31,32,32,33,33,16,34,34,35,36
4997,32,34,34,32,15,30,30,25,28,28,28,12,29,29,27,27,27,26,14,26,28,28,29,29,29,14,28,31,33,33,33,32,15,33,34,34,34,34,34,16,...,29,29,30,30,30,30,14,15,30,30,31,31,30,14,31,33,33,33,34,35,16,34,34,34,35,37,37,16,31,31,32,32,33,33,16,34,34,35,36,36
4998,34,34,32,15,30,30,25,28,28,28,12,29,29,27,27,27,26,14,26,28,28,29,29,29,14,28,31,33,33,33,32,15,33,34,34,34,34,34,16,31,...,29,30,30,30,30,14,15,30,30,31,31,30,14,31,33,33,33,34,35,16,34,34,34,35,37,37,16,31,31,32,32,33,33,16,34,34,35,36,36,35


In [81]:
df_b = pd.DataFrame(df_b_x)
df_b["Target"]  = df_b_y

In [98]:
df_ci = pd.DataFrame(df_ci_x)
df_ci["Target"]  = df_ci_y

In [99]:
df_cii = pd.DataFrame(df_cii_x)
df_cii["Target"]  = df_cii_y

Data Prepared !

##Proper X,Y Preperation for input 

In [18]:
df_a_x = np.reshape(df_a_x,(df_a_x.shape[0],df_a_x.shape[1],1))

In [19]:
df_a_x.shape

(5000, 100, 1)

##Building Model

In [20]:
df_a_x = df_a_x.astype(float)
df_a_y = df_a_y.astype(float)

###A 0

In [202]:
x = data["A"][0][0]
y = data["A"][0][1]

In [203]:
import keras

In [219]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout

In [220]:
model = Sequential()

In [221]:
model.add(LSTM(100,activation="relu",return_sequences=True,input_shape =(100,1)))

In [222]:
model.add(Dropout(0.2))

In [223]:
model.add(LSTM(100,activation="relu"))

In [224]:
model.add(Dropout(0.2))

In [225]:
model.add(Dense(1))

In [226]:
optimizer = keras.optimizers.Adam(learning_rate=0.001,clipnorm = 1.0)

In [227]:
model.compile(
    optimizer = optimizer,
    loss = "mean_squared_error",
    metrics = keras.losses.MeanAbsolutePercentageError()
)

In [228]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 100, 100)          40800     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 121,301
Trainable params: 121,301
Non-trainable params: 0
_________________________________________________________________


In [229]:
x = np.array(x)

In [230]:
y = np.array(y)

In [217]:
x = np.reshape(x,(x.shape[0],x.shape[1],1))

In [231]:
history = model.fit(x,y,epochs = 400,verbose  = 1,validation_split=0.1,batch_size=1024)

Epoch 1/400
1/1 [==============================] - 3s 3s/step - loss: 3950996.5000 - mean_absolute_percentage_error: 786.4131 - val_loss: 5488.7188 - val_mean_absolute_percentage_error: 98.6741
Epoch 2/400
1/1 [==============================] - 0s 349ms/step - loss: 562771.2500 - mean_absolute_percentage_error: 506.1982 - val_loss: 326526.4688 - val_mean_absolute_percentage_error: 611.9957
Epoch 3/400
1/1 [==============================] - 0s 305ms/step - loss: 183873.5781 - mean_absolute_percentage_error: 296.2865 - val_loss: 365345.7188 - val_mean_absolute_percentage_error: 772.9688
Epoch 4/400
1/1 [==============================] - 0s 327ms/step - loss: 1182752.8750 - mean_absolute_percentage_error: 508.1865 - val_loss: 3219591.5000 - val_mean_absolute_percentage_error: 2299.9277
Epoch 5/400
1/1 [==============================] - 0s 320ms/step - loss: 193487.3750 - mean_absolute_percentage_error: 301.2321 - val_loss: 51038.2656 - val_mean_absolute_percentage_error: 294.3194
Epoch 6/

##Hyperparameter Tuning

In [128]:
!rm -r forecasting

In [129]:
#!pip install keras-tuner

In [136]:
def build_model(hp):
  model = keras.Sequential()  #Model Definition
  model.add(keras.layers.LayerNormalization())
  for i in range(hp.Int('num_layers',1,2)):       #Select number of hidden layers from 2 to 20
    model.add(LSTM(units=hp.Int("units_"+str(i),min_value = 20,max_value = 500,step = 20),
                           activation = "relu",return_sequences = True,
                          )
              )
    model.add(Dropout(0.4))
  model.add(LSTM(units =hp.Int("units_secondlast",min_value = 20,max_value = 100,step = 20),
                           activation = "relu" )
  )
  model.add(Dropout(0.4))
  model.add(Dense(1,activation = "linear"))
  model.compile(
      optimizer = keras.optimizers.Adam(
          hp.Choice('learning_rate',[1e-1,1e-2,1e-3,1e-4]),
          clipnorm = 1.0
      ),
      loss = keras.losses.MeanAbsolutePercentageError(),
      metrics = keras.losses.MeanAbsolutePercentageError()
  )
  return model

In [137]:
from kerastuner.tuners import RandomSearch

In [138]:
tuner = RandomSearch(
    build_model,
    objective = "val_loss",
    max_trials = 3,
    executions_per_trial = 1,
    directory='forecasting',
    project_name='lstm'
)

INFO:tensorflow:Reloading Oracle from existing project forecasting/lstm/oracle.json


In [139]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 20, 'max_value': 500, 'step': 20, 'sampling': None}
units_secondlast (Int)
{'default': None, 'conditions': [], 'min_value': 20, 'max_value': 100, 'step': 20, 'sampling': None}
learning_rate (Choice)
{'default': 0.1, 'conditions': [], 'values': [0.1, 0.01, 0.001, 0.0001], 'ordered': True}


In [140]:
tuner.search(df_a_x,df_a_y,epochs =10,validation_split = 0.15,batch_size = 1024)

Trial 3 Complete [00h 00m 18s]
val_loss: 80.4598159790039

Best val_loss So Far: nan
Total elapsed time: 00h 01m 03s
INFO:tensorflow:Oracle triggered exit


In [142]:
tuner.results_summary()

Results summary
Results in forecasting/lstm
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
num_layers: 1
units_0: 160
units_secondlast: 80
learning_rate: 0.1
Score: nan
Trial summary
Hyperparameters:
num_layers: 2
units_0: 40
units_secondlast: 40
learning_rate: 0.01
units_1: 20
Score: nan
Trial summary
Hyperparameters:
num_layers: 1
units_0: 180
units_secondlast: 60
learning_rate: 0.001
units_1: 440
Score: 80.4598159790039


In [144]:
model = tuner.get_best_models()

NotFoundError: ignored

##Data Preprocessing in different style

In [271]:
##Creating a dictionary to store timesteps according to domain , inventory
data = {}
domains = df["Domain"].unique()
inventories = df["InventoryCode"].unique()
for domain in domains:
  data[domain] = {}
  for inventory in inventories:
    data[domain][inventory] = df.loc[(df["Domain"]==domain) & (df["InventoryCode"]==inventory)]

In [272]:
def preprocess(timeseries_data,n_features):
  series_data = list(timeseries_data["Usage"])
  end_idx = n_features
  print(end_idx)
  seq_x,seq_y = series_data[:end_idx],series_data[end_idx:]
  return np.array(seq_x),np.array(seq_y)

In [273]:
for domain in domains:
  for inventory in inventories:
    req_features = n_features[domain]
    print(domain,inventory)
    x,y = preprocess(data[domain][inventory],req_features)
    data[domain][inventory] = [x,y]

B 0
75
B 1
75
B 2
75
B 3
75
B 4
75
B 5
75
B 6
75
B 7
75
B 8
75
B 9
75
B 10
75
B 11
75
B 12
75
B 13
75
B 14
75
B 15
75
B 16
75
B 17
75
B 18
75
B 19
75
B 20
75
B 21
75
B 22
75
B 23
75
B 24
75
B 25
75
B 26
75
B 27
75
B 28
75
B 29
75
B 30
75
B 31
75
B 32
75
B 33
75
B 34
75
B 35
75
B 36
75
B 37
75
B 38
75
B 39
75
B 40
75
B 41
75
B 42
75
B 43
75
B 44
75
B 45
75
B 46
75
B 47
75
B 48
75
B 49
75
B 50
75
B 51
75
B 52
75
B 53
75
B 54
75
B 55
75
B 56
75
B 57
75
B 58
75
B 59
75
B 60
75
B 61
75
B 62
75
B 63
75
B 64
75
B 65
75
B 66
75
B 67
75
B 68
75
B 69
75
B 70
75
B 71
75
B 72
75
B 73
75
B 74
75
B 75
75
B 76
75
B 77
75
B 78
75
B 79
75
B 80
75
B 81
75
B 82
75
B 83
75
B 84
75
B 85
75
B 86
75
B 87
75
B 88
75
B 89
75
B 90
75
B 91
75
B 92
75
B 93
75
B 94
75
B 95
75
B 96
75
B 97
75
B 98
75
B 99
75
B 100
75
B 101
75
B 102
75
B 103
75
B 104
75
B 105
75
B 106
75
B 107
75
B 108
75
B 109
75
B 110
75
B 111
75
B 112
75
B 113
75
B 114
75
B 115
75
B 116
75
B 117
75
B 118
75
B 119
75
B 120
75
B 121
75
B 122
75
B 1

In [274]:
print(data["A"][0][0])

[86 83 86 81 84 86 80 81 84 86 86 84 86 84 86 85 86 86 86 86 81 82 85 86
 86 86 86 86 85 86 86 83 86 86 86 86 83 85 86 86 85 86 86 86 86 84 86 85
 86 86 86 86 84 86 86 86 86 86 86 86 86 86 86 86 85 85 85 86 75 75 75 75
 75 75 75 75 75 75 75 75 75 75 75 75 75 75 75 75 75 75 75 75 75 75 74 74
 74 74 74 74]


In [280]:
x = {}
y = {}

for domain in domains:
  x[domain] = []
  y[domain] = []
  for inventory in inventories:
    if len(data[domain][inventory][0]) == 0:
      break
    y_to_add = data[domain][inventory][1]
    x_to_add = data[domain][inventory][0]
    x[domain].append(x_to_add)
    y[domain].append(y_to_add)

In [281]:
x["A"]

[array([86, 83, 86, 81, 84, 86, 80, 81, 84, 86, 86, 84, 86, 84, 86, 85, 86,
        86, 86, 86, 81, 82, 85, 86, 86, 86, 86, 86, 85, 86, 86, 83, 86, 86,
        86, 86, 83, 85, 86, 86, 85, 86, 86, 86, 86, 84, 86, 85, 86, 86, 86,
        86, 84, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 85, 85, 85, 86,
        75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
        75, 75, 75, 75, 75, 75, 75, 75, 75, 74, 74, 74, 74, 74, 74]),
 array([32, 35, 39, 39, 36, 38, 36, 32, 34, 38, 38, 39, 40, 38, 35, 38, 41,
        37, 38, 41, 41, 37, 38, 40, 39, 36, 40, 42, 38, 41, 38, 39, 42, 41,
        43, 40, 41, 42, 41, 42, 44, 44, 40, 41, 42, 41, 46, 45, 45, 41, 41,
        43, 45, 46, 48, 46, 41, 42, 44, 42, 45, 46, 49, 42, 44, 44, 43, 44,
        47, 47, 41, 42, 43, 46, 45, 45, 43, 39, 42, 46, 46, 44, 46, 47, 43,
        44, 44, 40, 43, 44, 47, 38, 39, 44, 46, 48, 48, 46, 42, 46]),
 array([ 2,  2,  2,  4,  6,  7,  7,  7,  3,  6, 10, 13, 13, 13, 13,  8, 13,
        14, 14, 14, 14, 

In [282]:
x_a = np.array(x["A"])

In [288]:
y_a = np.array(y["A"])

In [285]:
x_a = np.reshape(x_a,(x_a.shape[0],x_a.shape[1],1))

In [286]:
x_a.shape

(250, 100, 1)

In [293]:
x_a = x_a.astype(float)

In [300]:
y_a = np.reshape(y_a,(y_a.shape[0],y_a.shape[1],1))

In [304]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(100, 1)))
model.add(keras.layers.RepeatVector(20))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(keras.layers.TimeDistributed(Dense(1)))
model.compile(optimizer=optim, loss='mse')

In [306]:
model.fit(x_a,y_a,verbose = 1,epochs = 400)

Epoch 1/400
8/8 [==============================] - 1s 163ms/step - loss: nan
Epoch 2/400
8/8 [==============================] - 1s 162ms/step - loss: nan
Epoch 3/400
8/8 [==============================] - 1s 159ms/step - loss: nan
Epoch 4/400
8/8 [==============================] - 1s 164ms/step - loss: nan
Epoch 5/400
8/8 [==============================] - 1s 158ms/step - loss: nan
Epoch 6/400
8/8 [==============================] - 1s 165ms/step - loss: nan
Epoch 7/400
8/8 [==============================] - 1s 171ms/step - loss: nan
Epoch 8/400
8/8 [==============================] - 1s 163ms/step - loss: nan
Epoch 9/400
8/8 [==============================] - 1s 155ms/step - loss: nan
Epoch 10/400
8/8 [==============================] - 1s 160ms/step - loss: nan
Epoch 11/400
8/8 [==============================] - 1s 164ms/step - loss: nan
Epoch 12/400
8/8 [==============================] - 1s 168ms/step - loss: nan
Epoch 13/400
8/8 [==============================] - 1s 164ms/step - loss: